# Live Video Streaming Using Python and OpenCV
This is a [Flask](http://flask.pocoo.org/)
web server that uses a streaming response
to provide a stream of video frames captured from a camera in
[MotionJPEG](https://en.wikipedia.org/wiki/Motion_JPEG).
This format is very simple and not the most efficient,
but has the advantage that all browsers support it natively and without any client-side scripting required

## Streaming Requests and Responses
When video streaming, you want to send an enormous amount of data to the client,
much more than you want to keep in memory.
But when you are generating the data on the fly as in video,
how do you send that back to the client without the roundtrip to the filesystem?

Streaming is a technique in which the server provides the response to a request in chunks.
This stream of data can be created and manipulated incrementally,
and provides an interface for reading or writing asynchronous chunks of data,
only a subset of which might be available in memory at any given time.

By default, HTTP streaming is disabled and HTTP request and response
payloads are written to a buffer in memory before they are processed.
You can change this behavior by enabling streaming.
With streaming enabled, request and response payloads are streamed without modification to the client app
(for responses) and the target endpoint (for requests). 

Streaming responses allow the page that made the original request to start working
with the response as soon as the first chunk of data is available,
and potentially use parsers that are optimized for streaming to progressively display the content.

* [Stream Your Way to Immediate Responses](https://developers.google.com/web/updates/2016/06/sw-readablestreams)
* [Streaming requests and responses](http://docs.apigee.com/api-services/content/enabling-streaming)

In [ ]:
%env CAMERA=opencv

## `base_camera.py`

In [ ]:
import time
import threading
try:
    from greenlet import getcurrent as get_ident
except ImportError:
    try:
        from thread import get_ident
    except ImportError:
        from _thread import get_ident


class CameraEvent(object):
    """An Event-like class that signals all active clients when a new frame is
    available.
    """
    def __init__(self):
        self.events = {}

    def wait(self):
        """Invoked from each client's thread to wait for the next frame."""
        ident = get_ident()
        if ident not in self.events:
            # this is a new client
            # add an entry for it in the self.events dict
            # each entry has two elements, a threading.Event() and a timestamp
            self.events[ident] = [threading.Event(), time.time()]
        return self.events[ident][0].wait()

    def set(self):
        """Invoked by the camera thread when a new frame is available."""
        now = time.time()
        remove = None
        for ident, event in self.events.items():
            if not event[0].isSet():
                # if this client's event is not set, then set it
                # also update the last set timestamp to now
                event[0].set()
                event[1] = now
            else:
                # if the client's event is already set, it means the client
                # did not process a previous frame
                # if the event stays set for more than 5 seconds, then assume
                # the client is gone and remove it
                if now - event[1] > 5:
                    remove = ident
        if remove:
            del self.events[remove]

    def clear(self):
        """Invoked from each client's thread after a frame was processed."""
        self.events[get_ident()][0].clear()


class BaseCamera(object):
    thread = None  # background thread that reads frames from camera
    frame = None  # current frame is stored here by background thread
    last_access = 0  # time of last client access to the camera
    event = CameraEvent()

    def __init__(self):
        """Start the background camera thread if it isn't running yet."""
        if BaseCamera.thread is None:
            BaseCamera.last_access = time.time()

            # start background frame thread
            BaseCamera.thread = threading.Thread(target=self._thread)
            BaseCamera.thread.start()

            # wait until frames are available
            while self.get_frame() is None:
                time.sleep(0)


## `camera_opencv.py`

In [ ]:
import cv2
#from base_camera import BaseCamera


class Camera(BaseCamera):
    video_source = 0

    @staticmethod
    def set_video_source(source):
        Camera.video_source = source

    @staticmethod
    def frames():
        camera = cv2.VideoCapture(Camera.video_source)
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        while True:
            # read current frame
            _, img = camera.read()

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', img)[1].tobytes()

## `camera_pi.py`

This cell should be disabled when the Jupyter isn't running on a Raspberry Pi.

>**NOTE:** Using Jupyter notebook you can click on a cell, press `esc` and then `r`.
That converts it to a "raw" cell.
You can use `esc` + `y` to convert it back a operating cell.

## `camera.py`

In [ ]:
import time
#from base_camera import BaseCamera

base = '/home/jeff/Jupyter-Notebooks/src/flask-video-streaming/'

class Camera(BaseCamera):
    """An emulated camera implementation that streams a repeated sequence of
    files 1.jpg, 2.jpg and 3.jpg at a rate of one frame per second."""
    imgs = [open(base + f + '.jpg', 'rb').read() for f in ['1', '2', '3']]

    @staticmethod
    def frames():
        while True:
            time.sleep(1)
            yield Camera.imgs[int(time.time()) % 3]

## `app.py`

In [ ]:
%env CAMERA=opencv

#!/usr/bin/env python
from importlib import import_module
import os
import jinja2
from flask import Flask, render_template, Response

# import camera driver
#if os.environ.get('CAMERA'):
#    Camera = import_module('camera_' + os.environ['CAMERA']).Camera
#else:
#    from camera import Camera

# Raspberry Pi camera module (requires picamera package)
# from camera_pi import Camera

app = Flask(__name__)

# creating a new template loader and then assigning it to the jinja_loader
# attribute on the Flask application.
# ChoiceLoader will search for a named template in the order of the loaders,
# stopping on the first match. 
my_loader = jinja2.ChoiceLoader([
    app.jinja_loader,
    jinja2.FileSystemLoader('/home/jeff/Jupyter-Notebooks/src/flask-video-streaming/templates'),
])
app.jinja_loader = my_loader

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')


def gen(camera):
    """Video streaming generator function."""
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == '__main__':
    app.run(host='0.0.0.0', port='9000', threaded=True)

## Sources
* [Flask Video Streaming Revisited](https://blog.miguelgrinberg.com/post/flask-video-streaming-revisited)
* [An introduction to the Flask Python web app framework](https://opensource.com/article/18/4/flask)